In [1091]:
import requests
import warnings
from tqdm import tqdm, trange
import streamlit as st
import pandas as pd 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import unittest
import tables
import pickle
import pyarrow.parquet as pq
import pyarrow as pa
import plotly.io as pio
import math
import investpy
import sklearn.preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import subprocess
import numpy as np
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [1092]:
pio.templates.default = "plotly_dark"

In [1110]:
oi = yf.Ticker("TSLA")

In [1111]:
oi.info

{'address1': '1 Tesla Road',
 'city': 'Austin',
 'state': 'TX',
 'zip': '78725',
 'country': 'United States',
 'phone': '512 516 8177',
 'website': 'https://www.tesla.com',
 'industry': 'Auto Manufacturers',
 'industryKey': 'auto-manufacturers',
 'industryDisp': 'Auto Manufacturers',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive, and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits; and non-warranty after-sales vehicle, used vehicles, body shop and parts, supercharging, retail merchandise, and vehicle insurance services. This segment also provides sedans and sport utility vehicles through direct and used vehic

In [1098]:
df = pd.read_csv('Multiples_Database.csv')
df = df.drop(columns = ['Unnamed: 0'])
df = df.dropna()
df

,Company,Market Cap,Rev Growth,NI Growth,EV/EBITDA,EV/Rev,EBITDA Margin,Operating Margin,Debt/Equity,PE,ROE,ROA,Industry,Sector,Name,Beta,Trailing PEG Ratio,Revenue,EBITDA
1,AOS,9447875584,-0.037,-0.089,-0.089,-0.089,0.20779,0.17793,7.471,17.950413,0.29544,0.14337,specialty-industrial-machinery,industrials,A. O. Smith Corporation,1.169,1.6306,3893799936,8.091000e+08
2,ABT,223866765312,0.049,0.146,0.146,0.146,0.26263,0.18749,37.589,16.893980,0.14840,0.06488,medical-devices,healthcare,Abbott Laboratories,0.722,2.3206,41216999424,1.082500e+10
3,ABBV,336816898048,0.038,-0.123,-0.123,-0.123,0.46153,0.28928,1174.815,80.084030,0.56407,0.07720,drug-manufacturers-general,healthcare,AbbVie Inc.,0.613,2.5528,55532998656,2.563000e+10
4,ACN,241422778368,0.026,0.236,0.236,0.236,0.17052,0.14597,14.127,32.353730,0.26675,0.11627,information-technology-services,technology,Accenture plc,1.245,2.9750,64896462848,1.106591e+10
5,ADBE,188515368960,0.106,0.233,0.233,0.233,0.38831,0.36834,41.788,35.094814,0.35355,0.15969,software-infrastructure,technology,Adobe Inc.,1.299,1.5182,20946999296,8.134000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,WDAY,72322744320,0.167,0.661,0.661,0.661,0.08089,0.05324,40.312,44.645320,0.21081,0.01426,software-application,technology,"Workday, Inc.",1.352,2.2769,7863000064,6.360000e+08
495,XEL,38244515840,-0.005,0.018,0.018,0.018,0.40035,0.25165,154.330,19.360464,0.10262,0.02564,utilities-regulated-electric,utilities,Xcel Energy Inc.,0.398,2.1530,13763000320,5.510000e+09
496,XYL,31847657472,0.013,0.413,0.413,0.413,0.20050,0.13498,19.840,36.013733,0.08081,0.04429,specialty-industrial-machinery,industrials,Xylem Inc.,1.050,2.4925,8424000000,1.689000e+09
499,ZBH,20092538880,0.040,0.597,0.597,0.597,0.33670,0.17997,53.611,22.783297,0.08713,0.04610,medical-devices,healthcare,"Zimmer Biomet Holdings, Inc.",1.021,1.3240,7595500032,2.557400e+09


In [1108]:
fig = go.Figure()
fig.add_trace(go.Histogram(x = ((df['Rev Growth']-df['Rev Growth'].mean())/df['Rev Growth'].std()), nbinsx=1000))

In [1109]:
fig = go.Figure()
fig.add_trace(go.Histogram(x = ((df['ROE']-df['ROE'].mean())/df['ROE'].std()), nbinsx=1000))

In [1112]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]  # First table contains S&P 500 tickers

# Keep only the columns of interest
sp500_table = sp500_table[["Symbol"]]
sp500_list = sp500_table['Symbol'].to_list()

data = []



for ticker in tqdm(sp500_list):
    stock = yf.Ticker(ticker)
    
    # Extract relevant financial metrics
    try:
        yearly_perf = stock.info["52WeekChange"]
        data.append(yearly_perf)
        
        
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
df = pd.DataFrame(data)
df

 12%|█▏        | 61/503 [00:43<05:24,  1.36it/s]

Error fetching data for BRK.B: '52WeekChange'


 15%|█▌        | 76/503 [00:53<04:34,  1.56it/s]

Error fetching data for BF.B: '52WeekChange'


100%|██████████| 503/503 [05:51<00:00,  1.43it/s]


,0
0,0.932109
1,-0.170599
2,0.179361
3,0.098390
4,0.066371
...,...
496,0.078288
497,0.129106
498,0.458259
499,-0.184638


In [1121]:
fig = go.Figure()
fig.add_trace(go.Histogram(x = ((df[0]-df[0].mean())/df[0].std()), nbinsx=600))
fig.update_xaxes(title_text = 'Standardized 52 Week Change')
fig.update_yaxes(title_text = 'Quantity')